In [101]:
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy

In [103]:
def get_files_in_directory(directory_path):
    """
    Returns a list of all files in the specified directory (non-recursive).

    INPUTS:
    directory_path: the directory in which to search for files

    OUTPUTS: 
    files: the files found in the directory
    """
    
    files = []
    for entry in os.listdir(directory_path):
        full_path = os.path.join(directory_path, entry)
        if os.path.isfile(full_path):
            files.append(entry)
    return files

In [105]:
def get_ssp_data():

    """
    get_ssp_data() takes files from the SSP_USGS directory and creates a compiled array with all sound speeds and corresponding depths.

    OUTPUTS:
    ssp_arr: a 2-d array of sound speed values in [m/s]
    z_ssp_arr: a 2-d array of depth values in [m]
    """
    
    ssp_file_names = get_files_in_directory('/Users/matthewaidan/Documents/pythonscripts/acoustic-models/SSP_USGS')
    ssp_list = []
    z_ssp_list = []
    for ssp_file in ssp_file_names:
        ssp_full_filename = '/Users/matthewaidan/Documents/pythonscripts/acoustic-models/SSP_USGS/' + ssp_file
        try:
            with open(ssp_full_filename, 'r') as file:
                content = file.read()
                content = content.split('\n')[3:-1]
                content = [item.split(' ') for item in content]
                converted_list = np.array([[float(item) for item in sublist] for sublist in content])
                z_ssp = converted_list[:,0] # [m] corresponding vertical coordinates for ssp
                ssp = converted_list[:,1] # [m/s] sound speed profile, https://doi.org/10.5066/P9Y1MSTN
                z_ssp_list.append(z_ssp)
                ssp_list.append(ssp)
        except FileNotFoundError:
            print(f"Error: {file} was not found.", file = ssp_filename)
    
    ssp_lengths = [len(item) for item in ssp_list]
    ssp_arr = np.zeros((len(ssp_list), np.max(ssp_lengths)))
    z_ssp_arr = ssp_arr.copy()
    
    for i1 in range(len(ssp_list)):
        ssp_arr[i1,:len(ssp_list[i1])] = ssp_list[i1]
        z_ssp_arr[i1,:len(z_ssp_list[i1])] = z_ssp_list[i1]
    
    ssp_arr[ssp_arr == 0] = np.nan
    z_ssp_arr[z_ssp_arr == 0] = np.nan

    return [ssp_arr, z_ssp_arr]

In [129]:
def create_topo(z_0, z_lims, r_max, f, roughness = 1, c_0 = 1500):

    """
    INPUTS:
    z_0: starting depth for terrain [m]
    z_lims: two elements giving ideal bounds for z [m]
    r_max: extent of topography [m]
    f: frequency of study [Hz]
    roughness: this is roughly equivalent to the std in depth [m]
    c_0: reference sound spped [m]
    """

    # get all of the depth_measurements to inform random generation
    h = c_0/f/3
    z_lims = np.array(z_lims)

    # generate a smoothly changing terrain
    topo = np.zeros((np.int64(r_max/h),))
    topo[0] = z_0
    for i1 in range(1,np.int64(r_max/h)):
        skew_factor = -1/roughness*np.sum((topo[i1-1] - z_lims)/(np.diff(z_lims))) # skew the distribution towards the center
        topo[i1] = scipy.stats.skewnorm.rvs(skew_factor, loc = topo[i1-1], scale = roughness)

    return topo

In [151]:
def create_mesh(topo, r_max, f, c_0 = 1500):

    """
    create_mesh() uses a given topography to create a mesh of points on which to calculate a field. It 
    does so by first discerning the angle of the topography with respect to the horizontal and 
    situating slices in r such that adjacent slices only differ by one mesh point in their intersection
    with the topography.
    
    INPUTS: 
    topo: [m] 1-d array where each index indicates an elevation or depth at a corresponding range
    r_max: [m] a float indicating the maximum range
    f: [Hz] the frequency of study
    c_0 = 1500: [m/s] the reference speed of sound in the medium

    OUTPUTS:
    r_mesh: [m] 1-d array of the range points of the mesh
    z_mesh: [m] 1-d array of the depth points of the mesh
    """
    
    h = c_0/f/3 # [m] 1/3rd of wavelength
    topo = h*np.round(topo/h) # discretize topography in units of h
    z_mesh = np.arange(0, np.max(topo)+2*h, h) # set z_mesh to include one point past maximum depth and one point 

    topo_new = [topo[0]] # initiate interpolated topography
    r_mesh = [0] # initiate r_mesh vector to append to
    i1 = 1 # iterator for original topography 
    i2 = 1 # iterator for interpolated topography

    # for each depth value in the original topography, interpolate points if the change in depth is greater than h. 
    while i2 < np.size(topo):
        if np.abs(topo[i2] - topo_new[i1-1]) >= h:
            starting_r = r_mesh[i1-1]
            dr = 0
            while r_mesh[i1-1] + dr < starting_r + h:
                dr = h**2/np.abs(topo[i2] - topo_new[i1-1])
                r_mesh = np.append(r_mesh, r_mesh[i1-1] + dr)
                topo_new = np.append(topo_new, topo_new[i1-1]+h*np.sign(topo[i2]-topo_new[i1-1]))
                i1 += 1
            r_mesh = np.append(r_mesh, starting_r + h)
            topo_new = np.append(topo_new, topo[i2])
            
        elif np.abs(topo[i2] - topo_new[i1-1]) == 0:
            topo_new = np.append(topo_new, topo[i2])
            r_mesh = np.append(r_mesh, r_mesh[i1-1] + h)
            dr = h
        else:
            raise ValueError("change in depth < h. 'topo' has not been discretized correctly!")

        i1 += 1
        i2 += 1
        
    print("Mesh created with %d points." % (np.size(r_mesh)*np.size(z_mesh)))
    
    return [r_mesh, z_mesh, topo_new]

In [155]:
def generate_physical_params(f, r_max, z_0, z_lims, roughness = 10, plot_flag = True):

    """
    generate_physical_params() generates all topography, mesh, sound speeds, and attenuation coefficients
    """
    topo = create_topo(z_0, z_lims, r_max, f, roughness)
    [r_mesh, z_mesh, topo] = create_mesh(topo, r_max, f)
    
    # using topo, get the angle of the bottom with respect to the surface
    theta = np.arctan(np.diff(topo)/np.diff(r_mesh)) # radians
    theta = np.append(theta, 0)
    
    # initiate alpha 
    alpha = 10e-3*np.ones((np.size(r_mesh), np.size(z_mesh)))

    # initiate rho
    rho = 1000*np.ones((np.size(r_mesh), np.size(z_mesh))) # add one more index to accomodate rho_1/rho_2 structure
    
    # assign ssp to each depth slice
    mesh_ssp = np.zeros((np.size(r_mesh), np.size(z_mesh)))
    for i_range in range(np.size(r_mesh)):
        ind_choice = np.random.choice(np.unique(np.where(z_ssp_arr > np.max(topo[i_range]))[0]))
        while np.any(ssp_arr[ind_choice,:] == 0):
            ind_choice = np.random.choice(np.unique(np.where(z_ssp_arr > np.max(topo[i_range]))[0]))
        mesh_ssp[i_range,:] = np.interp(z_mesh, z_ssp_arr[ind_choice,:], ssp_arr[ind_choice,:])

        # for each mesh point under topography, assign values associated with the bottom material
        mesh_ssp[i_range, z_mesh > topo[i_range]] = 1624
        alpha[i_range, z_mesh > topo[i_range]] = 20
        rho[i_range, z_mesh > topo[i_range]] = 1700

        # for each mesh point at surface, assign values associated with surface
        mesh_ssp[i_range, 0] = 343 # [m/s]
        alpha[i_range, 0] = 20 # [dB/lambda]
        rho[i_range,0] = 1.21 # [kg/m^3]

    rho = np.append(rho, 1700*np.ones((np.shape(rho)[0],1)), axis = 1)

    if plot_flag:
        fig, ax = plt.subplots(figsize = (15,5))
        im = ax.pcolormesh(r_mesh, z_mesh, np.transpose(mesh_ssp), vmin = 1480, vmax = 1520)
        ax.invert_yaxis()
        cbar = fig.colorbar(im, ax=ax)

    return [theta, alpha, rho, r_mesh, z_mesh, mesh_ssp, topo]